In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf
stock_symbol = "RELIANCE.NS" #NSE stocks usually end with .NS
# Download stock data
stk_data = yf.download(stock_symbol, start="2023-01-01", end="2023-07-01")

[*********************100%***********************]  1 of 1 completed


In [3]:
stk_data

Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2023-01-02,1180.586060,1182.006865,1167.890577,1168.715541,5316175
2023-01-03,1171.946655,1179.256896,1167.707271,1175.613232,7658932
2023-01-04,1154.301392,1173.780010,1152.216007,1171.923749,9264891
2023-01-05,1152.239014,1162.482516,1147.632911,1156.570169,13637099
2023-01-06,1162.711548,1167.776018,1154.186834,1158.013796,6349597
...,...,...,...,...,...
2023-06-23,1152.559814,1161.107441,1151.299395,1159.044969,6628570
2023-06-26,1143.760010,1152.857653,1140.620538,1149.007778,12641159


In [4]:
stk_data=stk_data[["Open","High","Low","Close"]]
#stk_data.to_csv("Tatacoffee13_21.csv")

In [5]:
#column="Close"

In [6]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (122, 4)


In [8]:
import pandas as pd
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [9]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

98
X_train length: (98, 4)
X_test length: (24, 4)
y_train length: (98, 4)
y_test length: (24, 4)


In [10]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [11]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [12]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [13]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -27.476716669556545
BIC:  -26.93207161726552

Order = 2
AIC:  -27.41656854402199
BIC:  -26.42978170952454

Order = 3
AIC:  -27.354583179996272
BIC:  -25.9198063191295

Order = 4
AIC:  -27.130751527370045
BIC:  -25.24200644312051

Order = 5
AIC:  -26.936483246802187
BIC:  -24.58765790907747

Order = 6
AIC:  -26.923233759003494
BIC:  -24.108078288005533

Order = 7
AIC:  -27.022824452233817
BIC:  -23.734946960694373

Order = 8
AIC:  -26.934103792972017
BIC:  -23.166966082443377

Order = 9
AIC:  -26.75304314323658
BIC:  -22.499956249582855

Order = 10
AIC:  -26.916093744118257
BIC:  -22.17021332732893



In [14]:
data1

,Open,High,Low,Close
0,0.856777,0.930052,0.968438,0.947163
1,0.895268,0.914508,0.967386,0.899507
2,0.874680,0.883549,0.878485,0.802175
3,0.789003,0.819690,0.852184,0.790799
4,0.797059,0.849612,0.889795,0.848566
...,...,...,...,...
117,0.802813,0.811917,0.873225,0.792568
118,0.746803,0.765285,0.811941,0.744028
119,0.720077,0.750000,0.806550,0.746304
120,0.762149,0.823446,0.836007,0.829858


In [15]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.327094,1,28
